In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
#DOCUMENTATION(general flow of code)
#1-> imported the libraries that we may use during the preprocessing or training
#2-> showed a single image to check the proper paths n working of it
#3-> printed the shpe of image height*width*3(height n width was in pixels and 3 for 3d ie rgb layers)
#4-> using ImageDataGenerator we scaled the images as it is important to scale them before putting it in CNN network
#5-> creation of training and validation datasets through folders by giving paths
#6-> creation of cnn model with conviolutional layers with filters (their kernel sizes) and  max pooling (their stride jump)
#7-> compilation of model before fiiting it adam optimizer,binary croosentropy(for loss fxn)
#8-> backpropagation and all the updation with 25(random) epochs we reached very good accuracy of 100 % in some epochs and 99 on some
#9-> testing the model initially on a single pic then through entire folder we counted how many in total were predicted correct 
#10-> test folder with subfolders (with mask(positive) and without mask(negative)) correct-(TP+FN)     wrong-(TN+FP)
#11-> calculation of confusion matrix[[tp tn],[fp.fn]] , precsion,accuracy,recall,f1_score

import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop

In [9]:
img=image.load_img("../input/face-mask-12k-images-dataset/Face Mask Dataset/Test/WithMask/45.png")

In [10]:
plt.imshow(img)

In [11]:
cv2.imread("../input/face-mask-12k-images-dataset/Face Mask Dataset/Test/WithMask/45.png")

In [12]:
cv2.imread("../input/face-mask-12k-images-dataset/Face Mask Dataset/Test/WithMask/45.png").shape

In [13]:
#generating train and testing dataset
train=ImageDataGenerator(rescale=1/255)
validation=ImageDataGenerator(rescale=1/255)

In [14]:
train_dataset=train.flow_from_directory('../input/face-mask-12k-images-dataset/Face Mask Dataset/Train',
                                        target_size = (64, 64),
                                        batch_size=32,
                                        class_mode='binary')

In [15]:
validation_dataset=train.flow_from_directory('../input/face-mask-12k-images-dataset/Face Mask Dataset/Validation',
                                        target_size=(64, 64),
                                        batch_size=32,
                                        class_mode='binary')

In [16]:
train_dataset.class_indices

In [17]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [18]:
#cnn.fit(x = train_dataset, validation_data = validation_dataset, epochs = 25)
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [19]:
cnn.fit(x = train_dataset, validation_data = validation_dataset, epochs = 25)

In [20]:
dir_path='../input/face-mask-12k-images-dataset/Face Mask Dataset/Test'

for i in os.listdir(dir_path):
    print(i)

In [23]:
#making a single prediction

test_image = image.load_img('../input/face-mask-12k-images-dataset/Face Mask Dataset/Test/WithMask/45.png', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
train_dataset.class_indices
if result[0][0] == 1:
  prediction = 'without mask'
else:
  prediction = 'with mask'

In [24]:
print(prediction)

In [25]:
dir_path='../input/face-mask-12k-images-dataset/Face Mask Dataset/Test/WithMask'
wm=0
wtm=0
for i in os.listdir(dir_path):
    img=image.load_img(dir_path+'//'+ i,target_size = (64, 64))
    X=image.img_to_array(img)
    X=np.expand_dims(X,axis=0)
    images=np.vstack([X])
    val=cnn.predict(images)
    if val==0:
        wm=wm+1
    else:
        wtm=wtm+1
    
    

In [28]:
#print("hello")
print(wm)# with mask predicted with mask #TRUE POSITIVE#
print(wtm)# without mask predicted wih mask#TRUE NEGATIVE#

In [29]:
dir_path='../input/face-mask-12k-images-dataset/Face Mask Dataset/Test/WithoutMask'
wkm=0
wktm=0
for i in os.listdir(dir_path):
    img=image.load_img(dir_path+'//'+ i,target_size = (64, 64))
    X=image.img_to_array(img)
    X=np.expand_dims(X,axis=0)
    images=np.vstack([X])
    val=cnn.predict(images)
    if val==0:
        wkm=wkm+1
    else:
        wktm=wktm+1

In [30]:
print(wkm)# without mask predicted with mask  #False positive#
print(wktm)# without mask predicted wihout mask #FALSE NEGATIVE#

In [35]:
tp=wm
tn=wtm
fp=wkm
fn=wktm
result=np.array([[tp, tn], [fp, fn]])#confusion matrix
result

In [46]:
precision=tp/(tp+fp)
accuracy=tp/(tp+tn)
recall=tp/(tp+fn)
f1_score=2*((precision*recall)/(precison+recall))
print(precison)
print(accuracy)
print(recall)
print(f1_score)